In [8]:
import torch 
import torchvision
from torch import nn,optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [9]:
data_csv=pd.read_csv('./data.csv',usecols=[1])

In [10]:
data_csv=data_csv.dropna()
dataset=data_csv.apply(lambda x : x /(x.max() - x.min() ) )
dataset=dataset.values
dataset=dataset.astype('float32')

In [11]:
dataset[0:0+2]

array([[0.21621622],
       [0.22779922]], dtype=float32)

In [12]:
dataX,dataY=[],[]
for i in range(len(dataset)-2):
    dataX.append(dataset[i:i+2])
    dataY.append(dataset[i+2])

In [15]:
dataX[12:18]

[array([[0.22200772],
        [0.24324325]], dtype=float32), array([[0.24324325],
        [0.27220076]], dtype=float32), array([[0.27220076],
        [0.26061776]], dtype=float32), array([[0.26061776],
        [0.24131274]], dtype=float32), array([[0.24131274],
        [0.28764477]], dtype=float32), array([[0.28764477],
        [0.32818532]], dtype=float32)]

In [16]:
dataY[12:18]

[array([0.27220076], dtype=float32),
 array([0.26061776], dtype=float32),
 array([0.24131274], dtype=float32),
 array([0.28764477], dtype=float32),
 array([0.32818532], dtype=float32),
 array([0.32818532], dtype=float32)]

In [17]:
train_size=int(len(dataX)*0.7)
test_size=len(dataX)-train_size
train_X=dataX[:train_size]
train_Y=dataY[:train_size]
test_X=dataX[train_size:]
test_Y=dataY[train_size:]

In [21]:
len(train_X)

99

In [18]:
train_X[0].shape

(2, 1)

In [19]:
train_x=torch.Tensor(train_X).reshape(-1,1,2)
train_y=torch.Tensor(train_Y).reshape(-1,1,1)
test_x=torch.Tensor(test_X).reshape(-1,1,2)
test_y=torch.Tensor(test_Y).reshape(-1,1,1)

In [ ]:
class LSTM_REG(nn.Module):
    def __init__(self,input_size,hidden_size,output_size=1,num_layers=2):
        super().__init__()
        self.rnn=nn.LSTM(input_size,hidden_size,num_layers)
        self.reg=nn.Linear(hidden_size,output_size)
        
    def forward(self,x):
        x,_=self.rnn(x)
        s,b,h=x.shape
        x=x.view(s*b,h)
        x=self.reg(x)
        x=x.view(s,b,-1)
        return x

In [23]:
train_x.shape

torch.Size([99, 1, 2])

In [ ]:
net=LSTM_REG(2,4)
loss=nn.MSELoss()
optimizer=optim.Adam(net.parameters(),lr=1e-2)

In [ ]:
for e in range(1000):
    x=train_x
    y=train_y
    l=loss(net(x),y)
    optimizer.zero_grad()
    l.backward()
    optimizer.step()
    if (e+1) % 100 ==0:
        print('epochs %d ,loss : %.5f ' % (e+1,l.item()))

In [22]:
train_x.shape

torch.Size([99, 1, 2])

In [24]:
train_y.shape

torch.Size([99, 1, 1])

### add h and c

In [ ]:
class LSTM_REG2(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2):
        super().__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers)
        self.reg = nn.Linear(hidden_size, output_size)
        self.state = None

    def forward(self, x, state):
        x, self.state = self.rnn(x, state)
        s, b, in_s = x.shape
        x = x.view(s * b, in_s)
        x = self.reg(x)
        x = x.view(s, b, -1)
        return x, self.state

In [ ]:
net2=LSTM_REG2(2,4,1)
loss=nn.MSELoss()
optimizer=optim.Adam(net2.parameters(),lr=1e-2)

In [ ]:
state=None
for e in range(2000):
    x=train_x
    y=train_y
    if state is not None:
        state=(state[0].detach(),state[1].detach())
    (y_hat,state)=net2(x,state)
    l=loss(y_hat,y)
    optimizer.zero_grad()
    l.backward()
    optimizer.step()
    if (e+1) % 100 ==0:
        print('epoch %d,loss : %.5f' % (e+1,l.item()))